<div style='background-color: orange'>
<a id="TableOfContents"></a>
    <h1 style='text-align: center'>
        <b><i>
            TABLE OF CONTENTS:
        </i></b></h1>
<li><a href='#imports'>Imports</a></li>
<li><a href="#acquire">Acquire</a></li>
<li><a href='#prepare'>Prepare</a></li>
<li><a href="#wrangle">Wrangle</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<div style='background-color: orange'>
<a id="imports"></a>
    <h1 style='text-align: center'>
        <b><i>
            Imports
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats
from scipy import stats

# .py files
import env
import wrangle as w

<div style='background-color: orange'>
<a id="acquire"></a>
    <h1 style='text-align: center'>
        <b><i>
            Acquire
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

Acquire everything from the vanilla superstore database via sql query

- Superstore Vanilla Shape:
    - Rows: 1734
    - Columns: 22

In [2]:
# Define the SQL query
query = '''
    SELECT 
        *
    FROM
        orders
        LEFT JOIN categories USING(`Category ID`)
        LEFT JOIN customers USING(`Customer ID`)
        LEFT JOIN products USING(`Product ID`)
        LEFT JOIN regions USING(`Region ID`)
    '''

In [3]:
# Create the SQL connection
url = env.get_db_url('superstore_db')

In [4]:
# Obtain the dataframe
superstore = pd.read_sql(query, url)
superstore.shape

(1734, 22)

In [5]:
# Test .py file functionality
acquire_superstore = w.acquire_superstore()
acquire_superstore.shape

(1734, 22)

<div style='background-color: orange'>
<a id="prepare"></a>
    <h1 style='text-align: center'>
        <b><i>
            Prepare
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>
<li><a href='#prepareinitial'>Initial</a></li>
<li><a href='#preparefix'>Things To Fix</a></li>
<li><a href='#preparedrop'>Drop Columns</a></li>
<li><a href='#preparefixcol'>Fix Columns</a></li>
<li><a href='#preparepy'>'.py' File Testing</a></li>

<a id='prepareinitial'></a>
<h3><b><i>
    Initial
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [6]:
# Check for null values
superstore.isna().sum()

Region ID        0
Product ID       0
Customer ID      0
Category ID      0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Sales            0
Quantity         0
Discount         0
Profit           0
Category         0
Sub-Category     0
Customer Name    0
Product Name     0
Region Name      0
dtype: int64

In [7]:
# Check dtypes
superstore.dtypes

Region ID          int64
Product ID        object
Customer ID       object
Category ID        int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Segment           object
Country           object
City              object
State             object
Postal Code      float64
Sales            float64
Quantity         float64
Discount         float64
Profit           float64
Category          object
Sub-Category      object
Customer Name     object
Product Name      object
Region Name       object
dtype: object

---

<a id='preparefix'></a>
<h3><b><i>
    Things To Fix
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

- Null Values
    - None
- Drop Columns
    - Region ID
        - ID unnecessary
    - Product ID
        - ID unnecessary
    - Customer ID
        - ID unnecessary
    - Category ID
        - ID unnecessary
- Fix Columns
    - Order Date
        - Object ==> datetime
    - Ship Date
        - Object ==> datetime
    - COLUMN NAMES
        - ALL LOWERCASE
        - ' ' WITH '_'
        - '-' WITH '_'

---

<a id='preparedrop'></a>
<h3><b><i>
    Drop Columns
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [8]:
# Identify columns to drop from database
columns_to_drop = [
    'Region ID',
    'Product ID',
    'Customer ID',
    'Category ID'
]

In [9]:
# Drop the columns and re-verify database shape
superstore = superstore.drop(columns=columns_to_drop)
superstore.shape

(1734, 18)

---

<a id='preparefixcol'></a>
<h3><b><i>
    Fix Columns
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [10]:
# Convert Object ==> datetime
superstore['Order Date'] = pd.to_datetime(superstore['Order Date'])
superstore['Ship Date'] = pd.to_datetime(superstore['Ship Date'])

In [11]:
# Verify dtype changes
superstore['Order Date'].dtype, superstore['Ship Date'].dtype

(dtype('<M8[ns]'), dtype('<M8[ns]'))

In [12]:
# Lowercase all column names
superstore.columns = superstore.columns.str.lower()

# Replace ' ' with '_'
superstore.columns = superstore.columns.str.replace(' ', '_')

# Replace '-' with '_'
superstore.columns = superstore.columns.str.replace('-', '_')

In [13]:
# Verify name changes
superstore.columns.to_list()

['order_id',
 'order_date',
 'ship_date',
 'ship_mode',
 'segment',
 'country',
 'city',
 'state',
 'postal_code',
 'sales',
 'quantity',
 'discount',
 'profit',
 'category',
 'sub_category',
 'customer_name',
 'product_name',
 'region_name']

---

<a id='preparepy'></a>
<h3><b><i>
    '.py' File Testing
</i></b></h3>
<li><a href='#prepare'>Prepare Top</a></li>

In [14]:
# Check that .py file works properly
prepare_superstore = w.prepare_superstore()
prepare_superstore.columns.to_list()

['order_id',
 'order_date',
 'ship_date',
 'ship_mode',
 'segment',
 'country',
 'city',
 'state',
 'postal_code',
 'sales',
 'quantity',
 'discount',
 'profit',
 'category',
 'sub_category',
 'customer_name',
 'product_name',
 'region_name']

<div style='background-color: orange'>
<a id="wrangle"></a>
    <h1 style='text-align: center'>
        <b><i>
            Wrangle
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [15]:
train, validate, test = w.wrangle_superstore()
train.shape, validate.shape, test.shape

train.shape:(970, 18)
validate.shape:(417, 18)
test.shape:(347, 18)


((970, 18), (417, 18), (347, 18))

<div style='background-color: orange'>
<a id="misc"></a>
    <h1 style='text-align: center'>
        <b><i>
            Miscellaneous
        </i></b></h1>
<li><a href='#TableOfContents'>Table of Contents</a></li>